# ノートブックの概要
SAR model によるシミュレーションと、EBCMによる情報拡散ダイナミクスの記述を比較する。

## 使用方法
1. Jupyter Notebook または Google Colab で開く
2. 各セルを順番に実行する
3. 必要に応じてデータを変更して再実行


In [ ]:
# 必要なライブラリをインポート
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sympy as sp
import random
import heapq
import sys
import copy
from math import comb
from matplotlib.gridspec import GridSpec
import seaborn as sns
import time
import datetime
import pandas as pd
import os
from sar simulation import initialize_simulation
from utils import load_results_np

# グラフ全体のフォント設定
plt.rcParams['font.family'] = 'serif'  # 全体のフォントをSerifに設定
plt.rcParams['mathtext.fontset'] = 'cm'  # 数式のフォントをComputer Modernに設定
plt.rcParams['mathtext.rm'] = 'serif'  # TeXの通常フォントをSerifに設定
plt.rcParams['font.size'] = 12  # デフォルトフォントサイズ
plt.rcParams['axes.labelsize'] = 14  # 軸ラベルのフォントサイズ
plt.rcParams['axes.titlesize'] = 16  # タイトルのフォントサイズ
plt.rcParams['legend.fontsize'] = 12  # 凡例のフォントサイズ
plt.rcParams['grid.color'] = 'gray'  # グリッドの色を薄い灰色に設定
plt.rcParams['grid.linestyle'] = ':'  # グリッドを点線に設定
plt.rcParams['grid.linewidth'] = 0.5  # グリッドの線幅を設定

In [ ]:
###  ネットワークの作成  ###

V = 10000  # ノード数
k_ave = 10 # 設定したい平均次数

# ランダムレギュラー（RR）グラフの生成
G = nx.random_regular_graph(k_ave, V)
network = 'RR'

# 次数分布と平均次数の計算
pk = np.zeros(G.number_of_nodes())
z = 0
for i in degree_list:
    pk[i] += 1
    z += i
    z2 += i**2

z /= V
pk /= V

In [ ]:
### シミュレーションのパラメーター ###
itr = 200
gamma = 1.0
t_pair = [1, 4]
p = 0.3
p_list = [p, 1-p]
rho0 = 1 / V

t_range = 100
time_err = 40
tmax = t_range + time_err

dlamb = 0.01
lambda_values = np.arange(0, 1 + dlamb, dlamb)

dalpha = 0.01
alpha_max = 1.1
alpha_values = np.arange(0, alpha_max + dalpha, dalpha)

In [ ]:
# 初期条件
dt = 1
time_steps = (t_range + 1) / dt
times_con = np.arange(0, time_steps * dt, dt)

# 2次元配列の準備
theta_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
q_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
sl_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
sh_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
aa_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
ab_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
ra_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))
rb_t_ana = np.zeros(((len(alpha_values), len(lambda_values), len(times_con))))

# ルンゲクッタ法で解を求める
for alpha_idx, alpha in enumerate(alpha_values):
    k_vals = np.arange(lowest_degree, highest_degree + 1)
    for lamb_idx, lamb in enumerate(lambda_values):
        y0 = [1, 1, 1 - rho0, 1 - rho0, rho0, rho0, 0, 0]
        result = runge_kutta4(sar_derivatives, y0, times_con, args=(lamb, gamma, rho0, alpha, k_vals, pk, z))
        theta, q, sl, sh, aa, ab, ra, rb = result.T

        # 結果を保存
        theta_t_ana[alpha_idx, lamb_idx, :] = theta
        q_t_ana[alpha_idx, lamb_idx, :] = q
        sl_t_ana[alpha_idx, lamb_idx, :] = sl
        sh_t_ana[alpha_idx, lamb_idx, :] = sh
        aa_t_ana[alpha_idx, lamb_idx, :] = aa
        ab_t_ana[alpha_idx, lamb_idx, :] = ab
        ra_t_ana[alpha_idx, lamb_idx, :] = ra
        rb_t_ana[alpha_idx, lamb_idx, :] = rb

In [ ]:
# データの読み込み
aa_results, ab_results, a_results, r_results = load_results_np(alpha_values, lambda_values, itr, t_range)

# 結果の形状を確認
print("Shapes:")
print(f'aa_results shape: {aa_results.shape}')
print(f'ab_results shape: {ab_results.shape}')
print(f'a_results shape: {a_results.shape}')
print(f'r_results shape: {r_results.shape}')


In [ ]:
aa_results_ave = np.mean(aa_results, axis=2) / p_list[0]/V
ab_results_ave = np.mean(ab_results, axis=2) / p_list[1]/V
a_results_ave = np.mean(a_results, axis=2) / V
r_results_ave = np.mean(r_results, axis=2) / V

aa_results_std = np.std(aa_results, axis=2) / p_list[0]/V
ab_results_std = np.std(ab_results, axis=2) / p_list[1]/V
a_results_std = np.std(a_results, axis=2) / V
r_results_std = np.std(r_results, axis=2) / V

In [ ]:
# 平均と標準偏差を格納する配列の初期化
shape = (len(alpha_values), len(lambda_values), t_range+1)
aa_results_ave_valid = np.zeros(shape)
ab_results_ave_valid = np.zeros(shape)
a_results_ave_valid = np.zeros(shape)
r_results_ave_valid = np.zeros(shape)

aa_results_std_valid = np.zeros(shape)
ab_results_std_valid = np.zeros(shape)
a_results_std_valid = np.zeros(shape)
r_results_std_valid = np.zeros(shape)

# 各 alpha と lambda の組み合わせについて計算
for alpha_idx, alpha in enumerate(alpha_values):
    for lamb_idx, lamb in enumerate(lambda_values):
        valid = []
        # 有効なイテレーションを抽出
        for iteration in range(itr):
            if r_results[alpha_idx, lamb_idx, iteration, -1] > 20:
                valid.append(iteration)
        
        if valid:
            # 有効なデータを抽出
            aa_valid = aa_results[alpha_idx, lamb_idx, valid, :]
            ab_valid = ab_results[alpha_idx, lamb_idx, valid, :]
            a_valid = a_results[alpha_idx, lamb_idx, valid, :]
            r_valid = r_results[alpha_idx, lamb_idx, valid, :]
            
            # 平均と標準偏差を計算し、Vとp_listで割る
            aa_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(aa_valid, axis=0) / V / p_list[0]
            ab_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(ab_valid, axis=0) / V / p_list[1]
            a_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(a_valid, axis=0) / V
            r_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(r_valid, axis=0) / V
            
            aa_results_std_valid[alpha_idx, lamb_idx, :] = np.std(aa_valid, axis=0) / V / p_list[0]
            ab_results_std_valid[alpha_idx, lamb_idx, :] = np.std(ab_valid, axis=0) / V / p_list[1]
            a_results_std_valid[alpha_idx, lamb_idx, :] = np.std(a_valid, axis=0) / V
            r_results_std_valid[alpha_idx, lamb_idx, :] = np.std(r_valid, axis=0) / V
        else:
            # 有効なイテレーションがない場合、全イテレーションを対象に計算
            aa_valid = aa_results[alpha_idx, lamb_idx, :, :]
            ab_valid = ab_results[alpha_idx, lamb_idx, :, :]
            a_valid = a_results[alpha_idx, lamb_idx, :, :]
            r_valid = r_results[alpha_idx, lamb_idx, :, :]
            
            # 平均と標準偏差を計算し、Vとp_listで割る
            aa_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(aa_valid, axis=0) / V / p_list[0]
            ab_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(ab_valid, axis=0) / V / p_list[1]
            a_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(a_valid, axis=0) / V
            r_results_ave_valid[alpha_idx, lamb_idx, :] = np.mean(r_valid, axis=0) / V
            
            aa_results_std_valid[alpha_idx, lamb_idx, :] = np.std(aa_valid, axis=0) / V / p_list[0]
            ab_results_std_valid[alpha_idx, lamb_idx, :] = np.std(ab_valid, axis=0) / V / p_list[1]
            a_results_std_valid[alpha_idx, lamb_idx, :] = np.std(a_valid, axis=0) / V
            r_results_std_valid[alpha_idx, lamb_idx, :] = np.std(r_valid, axis=0) / V

# 結果の確認
print("平均の形状:", aa_results_ave_valid.shape)
print("標準偏差の形状:", aa_results_std_valid.shape)

In [ ]:
overall_times = np.arange(0, t_range + 1, 1)
plt.figure(figsize=(18, 5))
plt.suptitle(rf'$Adoption Fraction: {network} (V={V}, T={tuple(t_pair)}itr={itr}, p={p}, \rho_0={int(rho0 * V)}nodes, \gamma={gamma})$')
colors = ['red', 'blue', 'green', 'purple']

alpha_idx_list = [0, 10, 42]
lamb_idx_list = [20, 28]

arg_idx = 0
for alpha_idx in alpha_idx_list:
    alpha = alpha_values[alpha_idx]
    plt.subplot(1, len(alpha_idx_list), arg_idx + 1)
    color_idx = 0
    for lamb_idx in lamb_idx_list:
        lamb = lambda_values[lamb_idx]
        # Aa の結果
        plt.errorbar(overall_times,
                    aa_results_ave_valid[alpha_idx, lamb_idx],
                    yerr=aa_results_std_valid[alpha_idx, lamb_idx],
                    capsize=5,
                    fmt='o',
                    markerfacecolor='none',
                    markeredgecolor=colors[color_idx], 
                    ecolor=colors[color_idx],
                    markersize=10,
                    label=rf'$\lambda$ = {lamb:.2f}, Aa')
        
        # Aa のEBCMによる近似
        plt.plot(times_con,
                    aa_t_ana[alpha_idx][lamb_idx],
                    linestyle='-',
                    linewidth=2,
                    color=colors[color_idx])

        # Ab の結果
        plt.errorbar(overall_times,
                    ab_results_ave_valid[alpha_idx, lamb_idx],
                    yerr=ab_results_std_valid[alpha_idx, lamb_idx],
                    capsize=5,
                    fmt='^',
                    markerfacecolor='none',
                    markeredgecolor=colors[color_idx + 1], 
                    ecolor=colors[color_idx + 1],
                    markersize=10,
                    label=rf'$\lambda$ = {lamb:.2f}, Ab')

        # Ab のEBCMによる近似
        plt.plot(times_con,
                    ab_t_ana[alpha_idx][lamb_idx],
                    linestyle='-',
                    linewidth=2,
                    color=colors[color_idx + 1])

        color_idx += 2
    plt.xlabel('Time Steps')
    plt.ylabel('A(t)')
    plt.xlim(0, 40)
    plt.ylim(0,0.25)
    plt.title(rf'$\alpha={alpha}$')
    plt.legend(loc='best')  # ラベルを表示
    plt.grid(True)  # グリッドを表示

    arg_idx+=1
plt.tight_layout()  # suptitle を考慮
plt.show()

In [ ]:
aa_max_indices = np.argmax(aa_t_ana, axis=2)
ab_max_indices = np.argmax(ab_t_ana, axis=2)

a_t_ana = p_list[0] * aa_t_ana + p_list[1] * ab_t_ana
r_t_ana = p_list[0] * ra_t_ana + p_list[1] * rb_t_ana

a_max_indices = np.argmax(a_t_ana, axis=2)
r_max_indices = np.argmax(r_t_ana, axis=2)

In [ ]:
# 平均と標準偏差を格納する配列の初期化
shape = (len(alpha_values), len(lambda_values), itr, t_range+1)
aa_results_align = np.zeros(shape)
ab_results_align = np.zeros(shape)
a_results_align = np.zeros(shape)
r_results_align = np.zeros(shape)

# 各時系列データをアラインメント
for alpha_idx in range(len(alpha_values)):
    for lamb_idx in range(len(lambda_values)):
        for i in range(itr):
            # aa_resultsのアラインメント
            aa_results_align[alpha_idx, lamb_idx, i, :] = align_to_max(
                aa_results[alpha_idx, lamb_idx, i, :],
                ref_index=aa_max_indices[alpha_idx, lamb_idx]
            )
            
            # ab_resultsのアラインメント
            ab_results_align[alpha_idx, lamb_idx, i, :] = align_to_max(
                ab_results[alpha_idx, lamb_idx, i, :],
                ref_index=ab_max_indices[alpha_idx, lamb_idx,]
            )
            
            # a_resultsのアラインメント
            a_results_align[alpha_idx, lamb_idx, i, :] = align_to_max(
                a_results[alpha_idx, lamb_idx, i, :],
                ref_index=a_max_indices[alpha_idx, lamb_idx]
            )
            
            # r_resultsのアラインメント
            r_results_align[alpha_idx, lamb_idx, i, :] = align_to_max(
                r_results[alpha_idx, lamb_idx, i, :],
                ref_index=r_max_indices[alpha_idx, lamb_idx]
            )


In [ ]:
t_max = 40 # 図示する時刻の最大値

In [ ]:
plt.figure(figsize=(18,6))
plt.suptitle(f'All simulation over time: {network}', fontsize=18)
colors = ['red', 'blue', 'green', 'purple', 'orange']
colors2=['red', 'blue', 'green', 'purple', 'orange']

for a, alpha_idx in enumerate(alpha_idx_list):
    alpha = alpha_values[alpha_idx]
    plt.subplot(1, len(alpha_idx_list), a+1)
    plt.title(rf'$\alpha={alpha}$')

    for l, lamb_idx in enumerate(lamb_idx_list):
        lamb = lambda_values[lamb_idx]
        valid_trials = [] # 条件を満たす試行のインデックスを保存
        color=colors[l]

        for iteration in range(itr):
            if iteration == 1:
                plt.plot([],[],linestyle='-',color=color,alpha=1,label=rf'$Aa(t): \lambda={lamb:.2f}$')

            plt.plot(np.arange(t_max + 1),
                    a_results[alpha_idx, lamb_idx, iteration, :t_max+1] / (V),
                    linestyle='-',
                    color=color,
                    alpha=0.08)

            # 条件を満たすかチェック
            # if np.max(r_sim) > threshold_ratio:
            #     valid_trials.append(iteration)
        
        print(f'alpha={alpha}, lamb={lamb}, itr={itr}, varid: {len(valid_trials)}')

        plt.plot(np.arange(t_max + 1), np.mean(a_results[alpha_idx, lamb_idx], axis=0)[:t_max+1] / V, label=rf'$Average R(t): \lambda={lamb:.2f}$', color=colors2[l], alpha=0.8)
        # plt.plot(times_con, a_t_ana[alpha_idx, lamb_idx], label=f'Analytival A(t): lamb={lamb}', color=colors2[lamb_idx], alpha=0.8,linestyle='--')
    plt.legend()
    plt.grid('True')
plt.show()


In [ ]:
plt.figure(figsize=(15,4))
plt.suptitle(f'All simulation over time: {network}', fontsize=18)
colors = ['red', 'blue', 'green', 'purple', 'orange']
markers = ['o', 's', '^']  # マーカーの種類

for a, alpha_idx in enumerate(alpha_idx_list):
    alpha = alpha_values[alpha_idx]
    plt.subplot(1, len(alpha_idx_list), a+1)
    plt.title(rf'$\alpha={alpha}$')

    for l, lamb_idx in enumerate(lamb_idx_list):
        lamb = lambda_values[lamb_idx]
        valid_trials = [] # 条件を満たす試行のインデックスを保存

        for iteration in range(itr):
            if iteration == 1:
                plt.plot([],[],linestyle='-',color=colors[2*l],alpha=1,label=rf'$Aa(t): \lambda={lamb:.2f}$')
                plt.plot([],[],linestyle='-',color=colors[2*l+1],alpha=1,label=rf'$Ab(t): \lambda={lamb:.2f}$')

            plt.plot(np.arange(t_max + 1),
                    aa_results_align[alpha_idx, lamb_idx, iteration, :t_max+1] / (p_list[0]*V),
                    linestyle='-',
                    color=colors[2*l],
                    alpha=0.01)
            plt.plot(np.arange(t_max + 1),
                    ab_results_align[alpha_idx, lamb_idx, iteration, :t_max+1] / (p_list[1]*V),
                    linestyle='-',
                    color=colors[2*l+1],
                    alpha=0.01)

            # 条件を満たすかチェック
            if r_results_align[alpha_idx, lamb_idx, iteration, -1] > 20:
                valid_trials.append(iteration)

        # 有効な試行の平均と標準偏差を計算してプロット
        if valid_trials:
            # Aa(t) の平均と標準偏差
            mean_aa = np.mean(aa_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[0] * V)
            std_aa = np.std(aa_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[0] * V)
            
            # Ab(t) の平均と標準偏差
            mean_ab = np.mean(ab_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[1] * V)
            std_ab = np.std(ab_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[1] * V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_aa, yerr=std_aa, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)
            plt.errorbar(np.arange(t_max + 1), mean_ab, yerr=std_ab, fmt=markers[0], markerfacecolor='none',color=colors[2*l+1], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l+1], alpha=0.5)
        else:
            # 有効な試行がない場合、全試行の平均と標準偏差をプロット
            mean_aa = np.mean(aa_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[0] * V)
            std_aa = np.std(aa_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[0] * V)
            
            mean_ab = np.mean(ab_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[1] * V)
            std_ab = np.std(ab_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[1] * V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_aa, yerr=std_aa, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)
            plt.errorbar(np.arange(t_max + 1), mean_ab, yerr=std_ab, fmt=markers[0], markerfacecolor='none',color=colors[2*l+1], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l+1], alpha=0.5)

        plt.plot(np.arange(t_max + 1), aa_t_ana[alpha_idx, lamb_idx, :t_max+1], color=colors[2*l])
        plt.plot(np.arange(t_max + 1), ab_t_ana[alpha_idx, lamb_idx, :t_max+1], color=colors[2*l+1])

        plt.legend()
        plt.tight_layout()
    plt.ylim(0,0.4)
    plt.grid('True')
plt.show()


In [ ]:
alpha_idx_list = [0, 6, 25]
lamb_idx_list = [16, 24]
t_max = 60

In [ ]:
plt.figure(figsize=(10,4))
plt.suptitle(f'All simulation over time: {network}', fontsize=18)
colors = ['red', 'blue', 'green', 'purple', 'orange']
markers = ['o', 's', '^']  # マーカーの種類

for a, alpha_idx in enumerate(alpha_idx_list):
    alpha = alpha_values[alpha_idx]
    plt.subplot(1, len(alpha_idx_list), a+1)
    plt.title(rf'$\alpha={alpha:.2f}$')

    for l, lamb_idx in enumerate(lamb_idx_list):
        lamb = lambda_values[lamb_idx]
        valid_trials = [] # 条件を満たす試行のインデックスを保存

        for iteration in range(itr):
            if iteration == 1:
                plt.plot([],[],linestyle='-',color=colors[2*l],alpha=1,label=rf'$Aa(t): \lambda={lamb:.2f}$')
                plt.plot([],[],linestyle='-',color=colors[2*l+1],alpha=1,label=rf'$Ab(t): \lambda={lamb:.2f}$')

            plt.plot(np.arange(t_max + 1),
                    aa_results_align[alpha_idx, lamb_idx, iteration, :t_max+1] / (p_list[0]*V),
                    linestyle='-',
                    color=colors[2*l],
                    alpha=0.01)
            plt.plot(np.arange(t_max + 1),
                    ab_results_align[alpha_idx, lamb_idx, iteration, :t_max+1] / (p_list[1]*V),
                    linestyle='-',
                    color=colors[2*l+1],
                    alpha=0.01)

            # 条件を満たすかチェック
            if r_results_align[alpha_idx, lamb_idx, iteration, -1] > 20:
                valid_trials.append(iteration)

        # 有効な試行の平均と標準偏差を計算してプロット
        if valid_trials:
            # Aa(t) の平均と標準偏差
            mean_aa = np.mean(aa_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[0] * V)
            std_aa = np.std(aa_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[0] * V)
            
            # Ab(t) の平均と標準偏差
            mean_ab = np.mean(ab_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[1] * V)
            std_ab = np.std(ab_results_align[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (p_list[1] * V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_aa, yerr=std_aa, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)
            plt.errorbar(np.arange(t_max + 1), mean_ab, yerr=std_ab, fmt=markers[0], markerfacecolor='none',color=colors[2*l+1], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l+1], alpha=0.5)
        else:
            # 有効な試行がない場合、全試行の平均と標準偏差をプロット
            mean_aa = np.mean(aa_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[0] * V)
            std_aa = np.std(aa_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[0] * V)
            
            mean_ab = np.mean(ab_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[1] * V)
            std_ab = np.std(ab_results_align[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (p_list[1] * V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_aa, yerr=std_aa, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)
            plt.errorbar(np.arange(t_max + 1), mean_ab, yerr=std_ab, fmt=markers[0], markerfacecolor='none',color=colors[2*l+1], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l+1], alpha=0.5)

        plt.plot(np.arange(t_max + 1), aa_t_ana[alpha_idx, lamb_idx, :t_max+1], color=colors[2*l])
        plt.plot(np.arange(t_max + 1), ab_t_ana[alpha_idx, lamb_idx, :t_max+1], color=colors[2*l+1])

        plt.legend()
        plt.tight_layout()
    plt.ylim(0,0.2)
    plt.grid('True')
plt.show()


In [ ]:
t_max=100

In [ ]:
plt.figure(figsize=(10,4))
plt.suptitle(f'All simulation over time: {network}', fontsize=18)
colors = ['red', 'blue', 'green', 'purple', 'orange']
markers = ['o', 's', '^']  # マーカーの種類

for a, alpha_idx in enumerate(alpha_idx_list):
    alpha = alpha_values[alpha_idx]
    plt.subplot(1, len(alpha_idx_list), a+1)
    plt.title(rf'$\alpha={alpha:.2f}$')

    for l, lamb_idx in enumerate(lamb_idx_list):
        lamb = lambda_values[lamb_idx]
        valid_trials = [] # 条件を満たす試行のインデックスを保存

        for iteration in range(itr):
            if iteration == 1:
                plt.plot([],[],linestyle='-',color=colors[2*l],alpha=1,label=rf'$R(t): \lambda={lamb:.2f}$')

            plt.plot(np.arange(t_max + 1),
                    r_results[alpha_idx, lamb_idx, iteration, :t_max+1] / (V),
                    linestyle='-',
                    color=colors[2*l],
                    alpha=0.01)

            # 条件を満たすかチェック
            if r_results[alpha_idx, lamb_idx, iteration, -1] > 20:
                valid_trials.append(iteration)

        # 有効な試行の平均と標準偏差を計算してプロット
        if valid_trials:
            # R(t) の平均と標準偏差
            mean_r = np.mean(r_results[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (V)
            std_r = np.std(r_results[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0) / (V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_r, yerr=std_r, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)
        else:
            # 有効な試行がない場合、全試行の平均と標準偏差をプロット
            mean_r = np.mean(r_results[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (V)
            std_r = np.std(r_results[alpha_idx, lamb_idx, :, :t_max+1], axis=0) / (V)
            
            # 標準偏差のエラーバーを追加
            plt.errorbar(np.arange(t_max + 1), mean_r, yerr=std_r, fmt=markers[0], markerfacecolor='none',color=colors[2*l], markersize=6,linestyle='',capsize=3,ecolor=colors[2*l], alpha=0.5)

        plt.plot(np.arange(t_max + 1), r_t_ana[alpha_idx, lamb_idx, :t_max+1], color=colors[2*l])

        plt.legend()
        plt.tight_layout()
    plt.ylim(0,1.0)
    plt.grid('True')
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.suptitle(f'Lately Fraction: {network}', fontsize=18)
plt.axvline(x=0.6, color='black',linewidth=1, linestyle='--',alpha=0.7)
colors = ['red', 'blue', 'green', 'purple', 'orange']
markers = ['o', 's', '^']  # マーカーの種類

for a, alpha_idx in enumerate(alpha_idx_list):
    alpha = alpha_values[alpha_idx]

    mean_r = np.zeros_like(lambda_values)
    for lamb_idx, lamb in enumerate(lambda_values):
        valid_trials = [] # 条件を満たす試行のインデックスを保存

        for iteration in range(itr):

            # 条件を満たすかチェック
            if r_results[alpha_idx, lamb_idx, iteration, -1] > 20:
                valid_trials.append(iteration)

        # 有効な試行の平均と標準偏差を計算してプロット
        if valid_trials:
            # R(t) の平均と標準偏差
            mean_r[lamb_idx] = np.mean(r_results[alpha_idx, lamb_idx, valid_trials, :t_max+1], axis=0)[-1] / (V)
            
        else:
            # 有効な試行がない場合、全試行の平均と標準偏差をプロット
            mean_r[lamb_idx] = np.mean(r_results[alpha_idx, lamb_idx, :, :t_max+1], axis=0)[-1] / (V)
            
    plt.plot(lambda_values, mean_r, color=colors[a], marker='o',markerfacecolor='none',markersize=10,linestyle='none',label=rf'$R({t_range}): \alpha={alpha:.2f}$')
    plt.plot(lambda_values, r_t_ana[alpha_idx, :, -1], color=colors[a])

plt.legend()
plt.xlabel(rf'$\lambda$')
plt.ylabel(rf'Fraction of R')
plt.tight_layout()
plt.ylim(0,1.0)
plt.grid('True')
plt.show()


In [ ]:
# Seabornスタイルを使用
sns.set_context("talk", font_scale=1.2)
sns.set_style("whitegrid")

# 図の準備
fig = plt.figure(figsize=(14, 7))
gs = GridSpec(1, 2, width_ratios=[1, 1], figure=fig)

# カラーマップの範囲を統一
vmin = 0
vmax = 1

# サブプロット1: シミュレーション結果
ax1 = fig.add_subplot(gs[0, 0])
im1 = ax1.imshow(
    r_results_ave_valid[:,:,-1],
    extent=[0.0, 1.0, 0.0, 1.1],
    origin='lower',
    aspect='auto',
    cmap='magma',
    vmin=vmin,
    vmax=vmax
)
ax1.set_title("Simulation", fontsize=18, weight='bold')
ax1.set_xlabel(r"$\lambda$", fontsize=16)
ax1.set_ylabel(r"$\alpha$", fontsize=16, weight='bold')
ax1.grid(linewidth=0.5, alpha=0.7)  # グリッドを細く

# サブプロット2: 理論結果
ax2 = fig.add_subplot(gs[0, 1])
im2 = ax2.imshow(
    r_t_ana[:,:,-1],
    extent=[0.0, 1.0, 0.0, 1.1],
    origin='lower',
    aspect='auto',
    cmap='magma',
    vmin=vmin,
    vmax=vmax
)
ax2.set_title("Analytical", fontsize=18, weight='bold')
ax2.set_xlabel(r"$\lambda$", fontsize=16, weight='bold')
ax2.set_ylabel("")
ax2.grid(linewidth=0.5, alpha=0.7)  # グリッドを細く

# # カラーバーを全体で共有
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.67])  # 位置: [left, bottom, width, height]
cbar = fig.colorbar(im2, cax=cbar_ax)
cbar.set_label(rf"$R({t_range})$", fontsize=16, weight='bold')
cbar.ax.tick_params(labelsize=12)

# 全体のタイトル
plt.suptitle(rf"$R({t_range}): {network}, V={V}, <k>={z}, \gamma={gamma}, \rho_0={rho0}$", fontsize=20, weight='bold', y=0.95)

# レイアウト調整
plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.show()
